In [ ]:
import numpy as np
import pandas as pd
import nltk
import time
import openai
openai.api_key = "TYPE HERE YOUR API KEY"

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Determination of expected salary using NLTK


In [ ]:
# load data and make new columns
df = pd.read_csv('LI_PM_jobDesc.csv', delimiter = ',')
df['expectedSalary'] = ''

# definition of keywords
relevant_words = ['salary', 'payment', 'fizetés', 'ft', 'euro', 'dollar', 'font', '£', '$', '€']

# tokenization the jobDescriptions
for index, row in df.iterrows():
    tokens = nltk.word_tokenize(row['jobDescription'].lower())

    # determining whether the expected salary is given in the text
    for token in tokens:
        if token in relevant_words:
            df.loc[index, 'expectedSalary'] = 'salary_given' 
            break

# print results
df[['companyName', 'expectedSalary']]


## Determination of expected salary using AI


In [ ]:
# Létrehozzuk az új dataframe-et
LI_PM_jobad = df = pd.read_csv('LI_PM_jobDesc.csv', delimiter = ',')
df2 = pd.DataFrame()
df2['companyName'] = LI_PM_jobad['companyName']
df2['jobDescription'] = LI_PM_jobad['jobDescription']
df2['expectedSalary'] = ''

#iterate through the dataframe
for index, row in df2.iterrows():
    
    prompt_expectedSalary = row['jobDescription'] + "/n --- /n This is a product manager job description.\
                 Define the expected salary of this job, and\
                 write the value of salary into the dataframe (write just numbers and currency)\
                 Answer the question as truthfully as possible using the provided context.\
                 If the expected salary cannot be clearly determined based on the description,\
                 use just this value: 'Unknown'. Please, don't write 'Expected Salary:' into the dataframe."
        
    response_expectedSalary = openai.ChatCompletion.create(
        model = "gpt-4",
        messages = [
            {"role": "system", "content": "You are a helpful data science assistant."},
            {"role": "user", "content": prompt_expectedSalary}],
        max_tokens=50,
        temperature=0)
    
    message_expectedSalary = response_expectedSalary['choices'][0]['message']['content'].strip()

    df2.loc[index, 'expectedSalary'] = message_expectedSalary
    
    time.sleep(2)
        
    if index % 10 == 0:
        print(f"Processed row {index} for company {row['companyName']} with expected salary: {message_expectedSalary}")
        
#print results
df2[['companyName', 'expectedSalary']]